In [1]:
import pandas as pd

In [80]:
births = pd.read_csv("data/birth_data.csv")
births.columns = ["Month", "Birth Rate"]
births.index = births["Month"]
births.drop(columns=["Month"], inplace=True)


jobs = pd.read_csv("data/Employment_no_SA.csv")
jobs = jobs.drop(["Month.1"], axis=1)
jobs.index = jobs.Month
jobs = jobs.drop(["Month"], axis=1)
jobs.columns = ["Employment %"]

bus_rides = pd.read_csv("data/passengers_per_month.csv")
bus_rides.columns = ["Month", "Avg Bus Utilization"]
bus_rides.index = bus_rides["Month"]
bus_rides.drop(columns=["Month"], inplace=True)


population = pd.read_csv("data/population_data.csv")
air = pd.read_csv("data/air_quality_data.csv")
air["DateTime"] = pd.to_datetime(air["DateTime"])
air.set_index("DateTime", inplace=True)

air_month = pd.DataFrame(air.resample("BMS").mean()["AQI_Site"])
air_month.columns = ["AQI"]
air_month.index = pd.to_datetime(air_month.index)

Serieses = [births, jobs]
# Serieses = [births, jobs, bus_rides]
for s in Serieses:
    print(s.head(3))

            Birth Rate
Month                 
2006-01-01         390
2006-02-01         364
2006-03-01         417
            Employment %
Month                   
1978-02-01          61.1
1978-03-01          61.1
1978-04-01          61.1


In [81]:
doo = pd.concat(Serieses, axis=1)


/home/metaten/.virtualenvs/govhack/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [86]:
doo.dropna()
birth_employ = doo["Birth Rate"].corr(doo["Employment %"])

In [87]:
birth_employ

-0.35459212262231227